## Problem Statement

Gorverning Equation : du/dx=du/dt
boundary condition: u(x,0)=a*e^(b*x) (0 < a , b < 0)

- Independent variables: x,t (input)
- Dependent variables: u (outputs)


We have to find out u(x,t) for all x in range [0,2] and t in range [0,1]

In tranditional PINNs, a and b is fixed
But we give Boundary condtion to Neural Network, then model can find u each a and b

When we solved this problem analytically, we found the solution: u(x,t) = a*e^(-b*(x+t))


Our f is f = du/dx - du/dt

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import numpy as np


In [4]:
# We consider Net as our solution u_theta(x,t)

"""
When forming the network, we have to keep in mind the number of inputs and outputs
In ur case: #inputs = 2 + bc mesh number (x,t,bc)
and #outputs = 1
"""

class Net(nn.Module):
    def __init__(self,n_bc,bc_size):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(2+bc_size*3,bc_size*3)
        self.hidden_layer2 = nn.Linear(bc_size*3,bc_size*2)
        self.hidden_layer3 = nn.Linear(bc_size*2,bc_size)
        self.hidden_layer4 = nn.Linear(bc_size,50)
        self.hidden_layer5 = nn.Linear(50,10)
        self.output_layer = nn.Linear(10,1)

    def forward(self, x,t,bc):
        form_change_bc=bc.view(-1).repeat(x.shape[0],1)
        inputs = torch.cat([x,t,form_change_bc],axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer3(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer4(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer5(layer4_out))
        output = self.output_layer(layer5_out) ## For regression, no activation is used in output layer
        return output

In [5]:
## PDE as loss function. Thus would use the network which we call as u_theta
def f(x,t,bc, net):
    u = net(x,t,bc) # the dependent variable u is given by the network based on independent variables x,t and Boundary condition bc
    ## Based on our f = du/dx - du/dt, we need du/dx and du/dt
    u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    pde = u_x - u_t
    return pde

In [6]:
def merge_bc(x,t,u):
    col_list=[]
    for i in range(x.shape[1]):
        col_list.append(torch.stack([x[:,i],t[:,i],u[:,i]],dim=1))
    return torch.stack(col_list,0)

In [8]:
n_bc = 1000
bc_size = 200
x_bc = np.random.uniform(low=0.0, high=2.0, size=(bc_size,n_bc))
t_bc = np.zeros((bc_size,n_bc))
# compute u based on BC
a = np.random.uniform(low = 1.0, high = 6.0, size=(n_bc,1))
b = np.random.uniform(low = -4.0, high = -0.1, size=(n_bc,1))
u_bc = np.transpose(a*np.exp(b*np.transpose(x_bc)))

In [9]:
### (2) Model
net = Net(n_bc,bc_size)
net = net.to(device)
mse_cost_function = torch.nn.MSELoss() # Mean squared error
optimizer = torch.optim.Adam(net.parameters())

In [10]:
### (3) Training / Fitting
iterations = 100000
previous_validation_loss = 99999999.0
for epoch in range(iterations):
    optimizer.zero_grad() # to make the gradients zero

    # Loss based on boundary conditions
    pt_x_bc = Variable(torch.from_numpy(x_bc).float(), requires_grad=False).to(device)  # 500,n_bc
    pt_t_bc = Variable(torch.from_numpy(t_bc).float(), requires_grad=False).to(device)  # 500,n_bc
    pt_u_bc = Variable(torch.from_numpy(u_bc).float(), requires_grad=False).to(device)  # 500,n_bc
    
    m_bc = merge_bc(pt_x_bc,pt_t_bc,pt_u_bc)    # n_bc,500,3

    # Choose BC
    random_index = torch.randint(low=0,high=n_bc-1,size=(1,))  
    random_bc = m_bc[random_index].view([m_bc.shape[1],m_bc.shape[2]])
    
    net_bc_out = net(random_bc[:,0].view(-1,1), random_bc[:,1].view(-1,1),random_bc) # output of u(x,t)
    mse_u = mse_cost_function(net_bc_out, random_bc[:,2].view(-1,1))
    
    # Loss based on PDE
    x_collocation = np.random.uniform(low=0.0, high=2.0, size=(bc_size,1))
    t_collocation = np.random.uniform(low=0.0, high=1.0, size=(bc_size,1))
    all_zeros = np.zeros((bc_size,1))
    
    
    pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(torch.from_numpy(t_collocation).float(), requires_grad=True).to(device)
    pt_all_zeros = Variable(torch.from_numpy(all_zeros).float(), requires_grad=False).to(device)
    
    f_out = f(pt_x_collocation, pt_t_collocation, random_bc, net) # output of f(x,t)
    mse_f = mse_cost_function(f_out, pt_all_zeros)
    
    # Combining the loss functions
    loss = mse_u + mse_f
    
    
    loss.backward()
    optimizer.step()

    with torch.autograd.no_grad():
    	print(epoch,"Traning Loss:",loss.data)
    

0 Traning Loss: tensor(7.4238)
1 Traning Loss: tensor(1.8828)
2 Traning Loss: tensor(0.4013)
3 Traning Loss: tensor(6.7789)
4 Traning Loss: tensor(4.8211)
5 Traning Loss: tensor(2.0734)
6 Traning Loss: tensor(2.1497)
7 Traning Loss: tensor(3.8520)
8 Traning Loss: tensor(1.2641)
9 Traning Loss: tensor(6.4613)
10 Traning Loss: tensor(8.4031)
11 Traning Loss: tensor(15.6119)
12 Traning Loss: tensor(1.0517)
13 Traning Loss: tensor(1.5956)
14 Traning Loss: tensor(5.6360)
15 Traning Loss: tensor(0.4041)
16 Traning Loss: tensor(2.6413)
17 Traning Loss: tensor(1.4098)
18 Traning Loss: tensor(4.4793)
19 Traning Loss: tensor(0.9525)
20 Traning Loss: tensor(9.8095)
21 Traning Loss: tensor(1.8079)
22 Traning Loss: tensor(1.2274)
23 Traning Loss: tensor(1.7734)
24 Traning Loss: tensor(6.2924)
25 Traning Loss: tensor(0.8122)
26 Traning Loss: tensor(2.9447)
27 Traning Loss: tensor(0.1938)
28 Traning Loss: tensor(0.5424)
29 Traning Loss: tensor(1.4358)
30 Traning Loss: tensor(3.3980)
31 Traning Loss: 

KeyboardInterrupt: 

In [11]:
torch.save(net.state_dict(),'./net_save.pt')

In [12]:
# u = e^-(x+t)
a,b = 1,-1
bc_x_ab = np.linspace(0,2,bc_size)
bc_t_ab = np.zeros_like(bc_x_ab)
bc_u_ab = a*np.exp(bc_x_ab*1)

input_list=[]

for x,t,u in zip(bc_x_ab,bc_t_ab,bc_u_ab):
    input_list.append([x,t,u])

input_numpy = np.array(input_list).reshape(-1)

bc_input_tensor = torch.from_numpy(input_numpy).float()

# u = e^(-x-t)
x_tensor = torch.FloatTensor([0.5])
t_tensor = torch.FloatTensor([0.5])

In [13]:
net(x_tensor.view(1,1),t_tensor.view(1,1),bc_input_tensor.view(1,-1))

tensor([[3.7472]], grad_fn=<AddmmBackward0>)

In [94]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

x=np.arange(0,2,0.02)
t=np.arange(0,1,0.02)
ms_x, ms_t = np.meshgrid(x, t)
## Just because meshgrid is used, we need to do the following adjustment
x = np.ravel(ms_x).reshape(-1,1)
t = np.ravel(ms_t).reshape(-1,1)

pt_x = Variable(torch.from_numpy(x).float(), requires_grad=True).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=True).to(device)
pt_u = net(pt_x,pt_t)
u=pt_u.data.cpu().numpy()
ms_u = u.reshape(ms_x.shape)

surf = ax.plot_surface(ms_x,ms_t,ms_u, cmap=cm.coolwarm,linewidth=0, antialiased=False)
             
             

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'

<Figure size 640x480 with 0 Axes>

In [20]:
# Save Model
torch.save(net.state_dict(), "model_uxt.pt")